# Import

In [2]:
import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
import requests
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
import pickle
import gseapy as gp
import mygene
from IPython.display import display, HTML
import re
from collections import deque
from goatools.obo_parser import GODag

In [3]:
pd.set_option('display.width', None)      # No line-wrapping
pd.set_option('display.max_columns', None)  # Show all columns

# Prep

In [4]:
DISEASE = "BIPOLAR"
DISEASE_FOLDER = f"../output/{DISEASE}/"
RESULT_FOLDER = DISEASE_FOLDER + "leiden_results/"
DGIDB_DIRECTORY = f"../../Gen_Hypergraph/output/DGIDB_{DISEASE}/"
MSIGDB_DIRECTORY = "../../Gen_Hypergraph/output/MSigDB_Full/"

with open(DGIDB_DIRECTORY + "gene_to_index.json", "r") as file:
    gene_to_index_distinct = json.load(file)

In [5]:
index_to_gene_distinct = {v: k for k, v in gene_to_index_distinct.items()}

In [6]:
def communities_cutoff(communities, cutoff = 100):
    result = []
    for community in communities:
        if len(community) >= cutoff:
            result.append(community)

    return result, len(result)

def community_central_genes_by_num(G, community_nodes, weight="weight", top_n=20):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked[:top_n]]

def community_central_genes_by_score(G, community_nodes, weight="weight",score_cap = 1):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked if Z[u] >= score_cap]

def community_central_genes_by_pct(G, community_nodes, weight="weight",pct = 0.3):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    top = int(len(ranked)*pct)
    return [u for u in ranked[:top]]

In [ ]:
# # Combine indices from both layers, first DGIDB and then MSIGDB
# dgidb_rev = {index : gene for gene,index in dgidb.items()}
# msigdb_rev = {index : gene for gene,index in msigdb.items()}

# a_max = max(dgidb_rev.keys())
# b_min = min(msigdb_rev.keys())
# shift = (a_max + 1) - b_min   # ensures no overlap
# index_to_gene = {**dgidb_rev, **{k + shift: v for k, v in msigdb_rev.items()}}

# print(dgidb_rev)
# print(msigdb_rev)
# print(index_to_gene)

In [7]:
# Loading result graph and communities
with open(f"{RESULT_FOLDER}/result_communities_DGIDB.pkl", "rb") as f:
    communities = pickle.load(f)
with open(f"{RESULT_FOLDER}/result_graph_DGIDB.pkl", "rb") as f:
    graph = pickle.load(f)

In [8]:
# Checking communities
communities_greater_100, num_greater_100 = communities_cutoff(communities,100)
print(communities_greater_100, '\n', num_greater_100)
print(len(communities))

[[4, 6, 7, 9, 10, 11, 13, 14, 22, 24, 26, 27, 28, 30, 39, 40, 46, 48, 50, 51, 52, 55, 56, 58, 59, 60, 62, 65, 68, 75, 76, 77, 78, 80, 81, 84, 87, 88, 90, 93, 94, 95, 101, 102, 103, 105, 107, 108, 116, 120, 123, 124, 130, 135, 136, 143, 144, 145, 146, 149, 150, 153, 154, 155, 157, 160, 161, 163, 164, 165, 166, 167, 170, 171, 172, 174, 175, 176, 178, 183, 185, 186, 188, 191, 192, 194, 196, 197, 198, 199, 200, 202, 203, 208, 209, 212, 213, 214, 216, 219, 220, 222, 223, 224, 226, 227, 228, 229, 230, 234, 237, 239, 240, 241, 244, 248, 250, 251, 252, 256, 261, 266, 270, 271, 272, 273, 279, 280, 283, 284, 285, 286, 287, 290, 291, 292, 295, 299, 300, 302, 303, 304, 309, 312, 313, 314, 318, 319, 322, 324, 325, 328, 329, 332, 333, 334, 335, 339, 343, 344, 346, 348, 350, 351, 355, 357, 358, 360, 361, 363, 367, 370, 372, 374, 377, 380, 382, 384, 385, 386, 387, 389, 390, 391, 392, 394, 397, 398, 404, 405, 406, 407, 409, 411, 413, 415, 417, 418, 422, 423, 424, 428, 434, 435, 439, 440, 441, 442, 444,

In [9]:
# Update communities to include only the important genes, determined by a community-size_cap and z-score_cap
size_cap = 30
pct = 0.3
communities_selected = []
for community in communities:
    if (len(community) >= size_cap):
        important_nodes = community_central_genes_by_pct(graph,community,pct = pct)
        communities_selected.append(important_nodes)

In [26]:
print(communities_selected)
print(len(communities_selected))

for c in communities_selected:
    print(len(c))

[[3122, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3148, 3149, 3150, 3168, 3169, 3194, 3195, 3196, 3197, 3232, 3234, 3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3253, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 4, 6, 7, 9, 10, 11, 13, 14, 22, 24, 26, 27, 28, 30, 39, 40, 46, 48, 50, 51, 52, 55, 56, 58, 59, 60, 62, 65, 68, 75, 76, 77, 78, 80, 81, 84, 87, 88, 90, 93, 94, 95, 101, 102, 103, 105, 107, 108, 116, 120, 123, 124, 130, 135, 136, 143, 144, 145, 146, 149, 150, 153, 154, 155, 157, 160, 161, 163, 164, 165, 166, 167, 170, 171, 172, 174, 175, 176, 178, 183, 185, 186, 188, 191, 192, 194, 196, 197, 198, 199, 200, 202, 203, 208, 209, 212, 213, 214, 216, 219, 220, 222, 223, 224, 226, 227, 228, 229, 230, 234, 237, 239, 240, 241, 244, 248, 250, 251, 252, 256, 261, 266, 270, 271, 272, 273, 279, 280, 283, 284, 285, 286, 287, 290, 291, 292

In [11]:
# Convert index to ncbi
communities_ncbi = [list(map(index_to_gene_distinct.get, c)) for c in communities_selected]
print(communities_ncbi)

[['4885', '5338', '3000', '4738', '1043', '1934', '2844', '2845', '1789', '1998', '3273', '1980', '1979', '1982', '2027', '1365', '1332', '1327', '1397', '1743', '3222', '83', '520', '539', '2054', '2593', '1497', '3101', '81607', '4883', '85413', '5528', '54084', '55312', '6009', '1454', '100422959', '2981', '48', '64798', '6135', '6218', '2615', '1578', '391051', '28232', '10197', '1875', '3604', '2875', '23761', '6165', '58986', '482', '3578', '5052', '6203', '6132', '6234', '6156', '6168', '5594', '3791', '142', '1838', '2159', '6263', '6351', '1437', '2261', '5340', '6158', '2099', '51426', '3091', '9971', '4088', '7468', '5742', '552', '3156', '9682', '10298', '2064', '2906', '1906', '354', '2648', '1395', '5605', '146850', '5916', '695', '3480', '7015', '1956', '6607', '1232', '6555', '6300', '285', '3265', '64979', '920', '7398', '1119', '293', '23385', '3569', '2625', '2534', '7531', '7852', '368', '246', '23552', '4790', '1902', '5159', '2980', '3784', '1787', '92737', '20', 

In [12]:
# NCBI to HGNC symbol
communities_HGNC = []
missed = []


for community in communities_ncbi:
    mg = mygene.MyGeneInfo()
    entrez_ids = [str(e) for e in community]

    results = mg.querymany(
        entrez_ids,
        scopes="entrezgene",
        fields="symbol",
        species="human"
    )

    # Build a mapping: input ID -> symbol (or None)
    id_to_symbol = {}
    for r in results:
        q = str(r.get("query"))
        id_to_symbol[q] = r.get("symbol") if not r.get("notfound") else None

    # Preserve original order
    symbols = [id_to_symbol.get(str(e), None) for e in entrez_ids]
    communities_HGNC.append(symbols)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found no hit:	['3']
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string for

In [13]:
print(communities_HGNC)

[['NPTX2', 'PLD2', 'GUCY2D', 'NEDD8', 'CD52', 'EEF1B2P2', 'GPR21', 'GPR22', 'DNMT3B', 'ELF2', 'HRG', 'EIF4EP1', 'EIF4EBP2', 'EIF4G2', 'ENO3', 'CLDN3', 'COX5BP3', 'COX4I1', 'CRIP2', 'DLST', 'HOXC5', 'ACTG1P10', 'ATP5MC2P1', 'ATP5PO', 'STX2', 'GAMT', 'CTNS', 'HK3', 'NECTIN4', 'NPR3', 'SLC22A16', 'PPP2R5D', 'TSPEAR', 'RFK', 'RHEB', 'CSNK1E', 'MIR4268', 'GUCA2B', 'ACO1', 'DEPTOR', 'RPL11', 'RPS17', 'LRRC32', 'CYP3A51P', 'UOX', 'SLCO3A1', 'PSME3', 'E2F5', 'TNFRSF9', 'GPT', 'PISD', 'RPL35A', 'PGAP6', 'ATP1B2', 'IL9', 'PRDX1', 'RPS9', 'RPL8', 'RPS28', 'RPL30', 'RPL37A', 'MAPK1', 'KDR', 'PARP1', 'DTNB', 'F10', 'RYR3', 'CCL4', 'CSF2', 'FGFR3', 'PLG', 'RPL28', 'ESR1', 'POLK', 'HIF1A', 'NR1H4', 'SMAD3', 'NSD2', 'PTGS1', 'AVPR1A', 'HMGCR', 'KDM4A', 'PAK4', 'ERBB2', 'GRIN2D', 'EDN1', 'KLK3', 'KAT2A', 'CRHR2', 'MAP2K2', 'PIK3R6', 'RARG', 'BTK', 'IGF1R', 'TERT', 'EGFR', 'SMN2', 'CCR3', 'SLC10A2', 'MAPK12', 'ANGPT2', 'HRAS', 'MRPL36', 'CD4', 'USP1', 'CHKA', 'SLC25A6', 'NCSTN', 'IL6', 'GATA3', 'FYN', '

In [14]:
import math

def drop_nan_from_communities(communities):
    cleaned_communities = []
    total_dropped = 0

    for i, community in enumerate(communities):
        cleaned = []
        dropped = 0
        for g in community:
            if g is None or (isinstance(g, float) and math.isnan(g)):
                dropped += 1
            else:
                cleaned.append(g)
        cleaned_communities.append(cleaned)
        total_dropped += dropped
        print(f"Community {i}: dropped {dropped} NaN entries")

    print(f"\nTotal dropped across all communities: {total_dropped}")
    return cleaned_communities

In [15]:
communities_HGNC = drop_nan_from_communities(communities_HGNC)

Community 0: dropped 0 NaN entries
Community 1: dropped 1 NaN entries
Community 2: dropped 0 NaN entries
Community 3: dropped 0 NaN entries
Community 4: dropped 0 NaN entries
Community 5: dropped 0 NaN entries
Community 6: dropped 0 NaN entries
Community 7: dropped 0 NaN entries
Community 8: dropped 0 NaN entries
Community 9: dropped 0 NaN entries
Community 10: dropped 0 NaN entries
Community 11: dropped 0 NaN entries
Community 12: dropped 0 NaN entries
Community 13: dropped 0 NaN entries
Community 14: dropped 0 NaN entries
Community 15: dropped 0 NaN entries
Community 16: dropped 0 NaN entries
Community 17: dropped 0 NaN entries
Community 18: dropped 0 NaN entries
Community 19: dropped 0 NaN entries
Community 20: dropped 0 NaN entries
Community 21: dropped 0 NaN entries
Community 22: dropped 0 NaN entries
Community 23: dropped 0 NaN entries
Community 24: dropped 0 NaN entries
Community 25: dropped 0 NaN entries
Community 26: dropped 0 NaN entries
Community 27: dropped 0 NaN entries
Co

# Categoization Prep

### GO-slim

In [16]:
DATA_DIRECTORY = "../../data"
GO_OBO = f"{DATA_DIRECTORY}/GO/go-basic.obo"            # put the file in your working dir (or give full path)
GOSLIM_OBO = f"{DATA_DIRECTORY}/GO/goslim_generic.obo"  # swap to another slim if you prefer
GOSLIM_PIR_OBO = f"{DATA_DIRECTORY}/GO/goslim_pir.obo"  # swap to another slim if you prefer
GOSLIM_YEAST_OBO = f"{DATA_DIRECTORY}/GO/goslim_yeast.obo"
GOSLIM_AGR_OBO = f"{DATA_DIRECTORY}/GO/goslim_agr.obo"

In [17]:
# pip install goatools


# GO library
go = GODag(GO_OBO, optional_attrs={'relationship'})

# SLIM libraries
slim = GODag(GOSLIM_OBO)
slim_pir = GODag(GOSLIM_PIR_OBO)
slim_yeast = GODag(GOSLIM_YEAST_OBO)
slim_agr = GODag(GOSLIM_AGR_OBO)

slim_ids = set(slim.keys())
slim_pir_ids = set(slim_pir.keys())
slim_yeast_ids = set(slim_yeast.keys())
slim_agr_ids = set(slim_agr.keys())


slim_names = {sid: slim[sid].name for sid in slim_ids}

../../data/GO/go-basic.obo: fmt(1.2) rel(2025-10-10) 42,666 Terms; optional_attrs(relationship)
../../data/GO/goslim_generic.obo: fmt(1.2) rel(go/2025-10-10/subsets/goslim_generic.owl) 205 Terms
../../data/GO/goslim_pir.obo: fmt(1.2) rel(go/2025-10-10/subsets/goslim_pir.owl) 617 Terms
../../data/GO/goslim_yeast.obo: fmt(1.2) rel(go/2025-10-10/subsets/goslim_yeast.owl) 295 Terms
../../data/GO/goslim_agr.obo: fmt(1.2) rel(go/2025-10-10/subsets/goslim_agr.owl) 94 Terms


In [18]:
GO_RE = re.compile(r"(GO:\d{7})")

def get_goid(term: str):
    if isinstance(term, str):
        m = GO_RE.search(term)
        if m:
            return m.group(1)
    return None

def map_to_goslim(go_id,slim_ids_param):
    """Return all GO-Slim IDs that are ancestors (or self) of the GO term."""
    if not go_id or go_id not in go:
        return []
    node = go[go_id]
    parents = node.get_all_parents()
    if parents and not isinstance(next(iter(parents)), str):
        anc_ids = {node.id} | {t.id for t in parents}
    else:
        anc_ids = {node.id} | set(parents)
    return sorted(anc_ids & slim_ids_param)

In [19]:
from collections import deque

def get_go_ancestors_at_depth(go_id, depth, include_relations=("is_a", "part_of")):
    """
    Return the set of GO term IDs that are exactly `depth` steps above `go_id`.

    Parameters
    ----------
    go_id : str
        Starting GO term (e.g., "GO:0051310").
    depth : int
        Exact distance up the DAG to return (1 = immediate parents).
    include_relations : tuple[str]
        Relation types to traverse upward, e.g. ("is_a", "part_of", "regulates", ...).

    Returns
    -------
    set[str]
        GO IDs at exactly `depth` from `go_id`. Empty set if none.
    """
    if depth < 1:
        return set()
    if go_id not in go:
        return []

    # One hop function honoring relation filter
    def parent_ids(term):
        ids = set()
        if "is_a" in include_relations:
            ids.update(p.id for p in term.parents)  # direct is_a/part_of often merged here
        rel = getattr(term, "relationship", {}) or {}
        for r in include_relations:
            ids.update(rel.get(r, []))
        # ensure IDs exist in DAG
        return {pid for pid in ids if pid in go}

    # BFS by levels until we reach the target depth
    level = 0
    frontier = {go_id}
    visited = {go_id}

    while frontier and level < depth:
        next_frontier = set()
        for node in frontier:
            for pid in parent_ids(go[node]):
                if pid not in visited:
                    visited.add(pid)
                    next_frontier.add(pid)
        level += 1
        frontier = next_frontier

    # When loop ends: if level == depth, `frontier` is exactly-depth nodes
    return frontier if level == depth else set()


### KEGG

In [20]:
def build_kegg_name_to_id(species="hsa"):
    """Map KEGG pathway name -> 'hsaXXXXX' (species-specific)."""
    lines = requests.get(f"https://rest.kegg.jp/list/pathway/{species}").text.strip().splitlines()
    name_to_id = {}
    for ln in lines:
        pid, raw = ln.split("\t")
        pid = pid.replace("path:", "")  # e.g. hsa03010
        # strip " - Homo sapiens (human)" suffix
        name = re.sub(r"\s*-\s*Homo sapiens.*$", "", raw).strip()
        name_to_id[name.lower()] = pid
    return name_to_id

name_to_id = build_kegg_name_to_id("hsa")

In [21]:
def get_kegg_level2(hsa_id: str) -> str | None:
    """
    Return the KEGG Level 2 category for a pathway like 'hsa03040'.
    Example: get_kegg_level2("hsa03040") -> 'Transcription'
    """
    url = f"http://rest.kegg.jp/get/{hsa_id}"
    try:
        text = requests.get(url, timeout=10).text
    except Exception:
        return None

    for line in text.splitlines():
        if line.startswith("CLASS"):
            # CLASS line looks like: CLASS       Genetic Information Processing; Transcription
            parts = [p.strip() for p in line.split(";", maxsplit=2)]
            if len(parts) >= 2:
                return parts[1]
            elif len(parts) == 1:
                return parts[0].replace("CLASS", "").strip()
    return None

### Reactome

In [22]:
def build_reactome_level1_map(species="9606"):
    """
    Returns { 'R-HSA-xxxxx': ['Top Level Name', ...], ... } for the given species.
    species can be a taxonomy id ('9606') or a name ('Homo sapiens').
    """
    # ensure spaces are encoded if a name is used
    species_path = species.replace(" ", "+")
    url = f"https://reactome.org/ContentService/data/eventsHierarchy/{species_path}"
    r = requests.get(url, headers={"Accept": "application/json"}, timeout=60)
    r.raise_for_status()
    trees = r.json()  # list of trees, one per TopLevelPathway

    mapping = {}
    def walk(node, top_name):
        st_id = node.get("stId")
        if st_id:
            mapping.setdefault(st_id, set()).add(top_name)
        for child in node.get("children", []):
            walk(child, top_name)

    for top in trees:
        top_name = top["name"]  # level-1 category name
        walk(top, top_name)

    # sets -> sorted lists
    return {k: sorted(v) for k, v in mapping.items()}

# Example:
reactome_level1 = build_reactome_level1_map("9606")
  # -> ['Signal Transduction']

In [23]:
print(reactome_level1["R-HSA-9007101"])

['Vesicle-mediated transport']


# Run Enrichment Analysis

In [24]:
# term_size_cap = 20
term_score_cap = 0.001
percentage = 0.1
important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value"])

### GO

In [25]:
# GO Analysis; save terms with small size and high p-value
i = 0
unique_entries = set()
for community in communities_HGNC:
    # Gene Ontology enrichment
    enr_go = gp.enrichr(
        gene_list=community,
        gene_sets=['GO_Biological_Process_2023',
                'GO_Molecular_Function_2023',
                'GO_Cellular_Component_2023'],
        organism='Human',
        outdir=None # don't write to disk
    )
    go_df = enr_go.results
    go_df = go_df.sort_values('Adjusted P-value')
    print(f"Size of community: {len(community)}")

    # Filter by overlap percentage and adjusted p-value
    mask =  (go_df["Adjusted P-value"] < term_score_cap) & (go_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    filtered = go_df[mask].copy()
    
    # Categorization from GO-Slim
    filtered["GO_ID"] = filtered["Term"].apply(get_goid)
    filtered["Slim_IDs"] = filtered["GO_ID"].apply(lambda id: map_to_goslim(id, slim_ids_param=slim_yeast_ids))
    # filtered["Slim_IDs"] = filtered["GO_ID"].apply(lambda id: get_go_ancestors_at_depth(id, depth=2, include_relations=("is_a", "part_of")))
    
    # Get empty count
    empty_count = (filtered["Slim_IDs"].apply(len) == 0).sum()
    
    # Get slim names    
    filtered["Slim_Names"] = filtered["Slim_IDs"].apply(lambda ids: [go[i].name for i in ids])
    
    # Get unique slim names
    new_cate  = set().union(*filtered["Slim_Names"])
    print(new_cate)
    print(len(new_cate))
    unique_entries = unique_entries.union(new_cate)
    
    
    # print number of filtered terms
    print(f"Number of filtered terms: {len(filtered)}")
    print(f"Number of unmapped terms: {empty_count}")
    
    # Add results to important terms
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_terms = pd.concat([important_terms, filtered], ignore_index=True)

    display(HTML(filtered[['Term','Overlap','Adjusted P-value','GO_ID',"Slim_IDs","Slim_Names"]].head(10).to_html(max_cols=None)))
    i += 1

Size of community: 820
{'regulation of protein modification process', 'transmembrane transport', 'DNA damage response', 'cytoplasmic vesicle', 'ion binding', 'hydrolase activity', 'regulation of organelle organization', 'protein phosphorylation', 'transmembrane transporter activity', 'monoatomic ion transport', 'cytoskeletal protein binding', 'regulation of DNA metabolic process', 'signaling', 'response to chemical', 'cellular_component', 'cytoplasmic translation', 'peptidyl-amino acid modification', 'kinase activity', 'biological_process', 'enzyme regulator activity', 'response to oxidative stress', 'enzyme binding', 'membrane', 'regulation of transport', 'peptidase activity', 'transferase activity', 'molecular_function', 'regulation of cell cycle'}
28
Number of filtered terms: 410
Number of unmapped terms: 29


C:\Users\celem\AppData\Local\Temp\ipykernel_46656\2914499282.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  important_terms = pd.concat([important_terms, filtered], ignore_index=True)


,Term,Overlap,Adjusted P-value,GO_ID,Slim_IDs,Slim_Names
0,Protein Phosphorylation (GO:0006468),96/500,2.332562e-34,GO:0006468,"[GO:0006468, GO:0008150]","[protein phosphorylation, biological_process]"
1,Phosphorylation (GO:0016310),81/429,3.074747e-28,GO:0016310,[GO:0008150],[biological_process]
2,Regulation Of Cell Population Proliferation (GO:0042127),105/766,3.468963e-25,GO:0042127,[GO:0008150],[biological_process]
3,Positive Regulation Of Cell Population Proliferation (GO:0008284),75/483,1.080106e-20,GO:0008284,[GO:0008150],[biological_process]
3858,Protein Serine/Threonine Kinase Activity (GO:0004674),62/342,1.733933e-20,GO:0004674,"[GO:0003674, GO:0016301, GO:0016740]","[molecular_function, kinase activity, transferase activity]"
4,Transmembrane Receptor Protein Tyrosine Kinase Signaling Pathway (GO:0007169),55/284,2.399375e-19,GO:0007169,[GO:0008150],[biological_process]
5,Positive Regulation Of Intracellular Signal Transduction (GO:1902533),76/525,2.929231e-19,GO:1902533,[GO:0008150],[biological_process]
6,Protein Modification Process (GO:0036211),90/711,4.104745e-19,GO:0036211,[GO:0008150],[biological_process]
7,Positive Regulation Of Phosphorylation (GO:0042327),49/231,4.867055e-19,GO:0042327,[GO:0008150],[biological_process]
8,Positive Regulation Of Cellular Process (GO:0048522),80/594,1.463130e-18,GO:0048522,[GO:0008150],[biological_process]


Size of community: 109
{'cellular_component'}
1
Number of filtered terms: 2
Number of unmapped terms: 0


,Term,Overlap,Adjusted P-value,GO_ID,Slim_IDs,Slim_Names
935,MHC Protein Complex (GO:0042611),4/21,0.000175,GO:0042611,[GO:0005575],[cellular_component]
936,Lumenal Side Of Endoplasmic Reticulum Membrane (GO:0098553),4/27,0.000376,GO:0098553,[GO:0005575],[cellular_component]


...

Size of community: 12
set()
0
Number of filtered terms: 0
Number of unmapped terms: 0


,Term,Overlap,Adjusted P-value,GO_ID,Slim_IDs,Slim_Names


Size of community: 12
set()
0
Number of filtered terms: 0
Number of unmapped terms: 0


,Term,Overlap,Adjusted P-value,GO_ID,Slim_IDs,Slim_Names


KeyboardInterrupt: 

In [ ]:
print(f"Unique categories: {unique_entries}")
print(f"Number of unique categories: {len(unique_entries)}")

Unique categories: {'mRNA processing', 'glycoprotein biosynthetic process', 'protein modification by small protein conjugation or removal', 'ion binding', 'regulation of transport', 'vesicle organization', 'chromosome segregation', 'biological_process', 'response to chemical', 'cytokinesis', 'Golgi vesicle transport', 'nucleus', 'transmembrane transport', 'regulation of organelle organization', 'peptidase activity', 'chromatin organization', 'nucleolus', 'methyltransferase activity', 'organelle fusion', 'tRNA processing', 'cytoskeleton', 'protein transport', 'chromosome', 'ribosome', 'regulation of DNA metabolic process', 'rRNA binding', 'ribosomal small subunit biogenesis', 'molecular_function', 'proteolysis involved in protein catabolic process', 'cytoskeletal protein binding', 'cytoskeleton organization', 'mitochondrial translation', 'regulation of cell cycle', 'lipid binding', 'RNA binding', 'cytoplasmic vesicle', 'rRNA processing', 'membrane', 'nucleobase-containing compound trans

### KEGG

In [ ]:
# KEGG
i = 0
for community in communities_HGNC:
    enr_path = gp.enrichr(
        gene_list=community,
        gene_sets=['KEGG_2021_Human'],
        organism='Human',
        outdir=None
    )
    KEGG_df = enr_path.results
    KEGG_df = KEGG_df.sort_values('Adjusted P-value')

    # Filter by overlap percentage and adjusted p-value
    mask =  (KEGG_df["Adjusted P-value"] < term_score_cap) & (KEGG_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    filtered = KEGG_df[mask].copy()
    
    # Categorization from KEGG Level 2
    filtered["KEGG_ID"] = filtered["Term"].str.replace(r"\s*-\s*Homo sapiens.*$", "", regex=True).str.lower().map(name_to_id)
    filtered["Level_2_Category"] = filtered["KEGG_ID"].map(get_kegg_level2)
    
    # print number of filtered terms
    print(f"Number of filtered terms: {len(filtered)}")
    
    # Add results to important terms
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_terms = pd.concat([important_terms, filtered], ignore_index=True)

    print(f"Size of community: {len(community)}")
    display(HTML(filtered[['Term','Overlap','Adjusted P-value',"KEGG_ID","Level_2_Category"]].head(10).to_html(max_cols=None)))
    i += 1

### Reactome

In [ ]:
# Reactome enrichment
i = 0
for community in communities_HGNC:
    enr_path = gp.enrichr(
        gene_list=community,
        gene_sets=['Reactome_2022'],
        organism='Human',
        outdir=None
    )
    Reactome_df = enr_path.results
    Reactome_df = Reactome_df.sort_values('Adjusted P-value')

    # Filter by overlap percentage and adjusted p-value
    mask =  (Reactome_df["Adjusted P-value"] < term_score_cap) & (Reactome_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    filtered = Reactome_df[mask].copy()
    
    # Categorization from Reactome Level 1
    filtered["Level_1_Categories"] = filtered["Term"].str.extract(r"(R-[A-Z]+-\d+)", expand=False).map(reactome_level1)
    
    # print number of filtered terms
    print(f"Number of filtered terms: {len(filtered)}")
    
    # Add results to important terms
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_terms = pd.concat([important_terms, filtered], ignore_index=True)

    print(f"Size of community: {len(community)}")
    display(HTML(filtered[['Term','Overlap','Adjusted P-value',"Level_1_Categories"]].head(10).to_html(max_cols=None)))
    i += 1

Number of filtered terms: 79
Size of community: 1265


,Term,Overlap,Adjusted P-value,Level_1_Categories
0,Cell Cycle R-HSA-1640170,141/654,1.386214e-36,[Cell Cycle]
1,mRNA Splicing - Major Pathway R-HSA-72163,72/181,1.489595e-36,[Metabolism of RNA]
2,Processing Of Capped Intron-Containing Pre-mRNA R-HSA-72203,82/242,6.843764e-36,[Metabolism of RNA]
3,mRNA Splicing R-HSA-72172,72/189,2.328317e-35,[Metabolism of RNA]
4,M Phase R-HSA-68886,102/380,4.267012e-35,[Cell Cycle]
5,"Signaling By Rho GTPases, Miro GTPases And RHOBTB3 R-HSA-9716542",132/660,2.989369e-31,[Signal Transduction]
6,Signaling By Rho GTPases R-HSA-194315,129/644,1.238772e-30,[Signal Transduction]
7,Mitotic Prometaphase R-HSA-68877,66/186,1.825688e-30,[Cell Cycle]
8,"Cell Cycle, Mitotic R-HSA-69278",114/523,1.825688e-30,[Cell Cycle]
9,Metabolism Of RNA R-HSA-8953854,118/666,4.445735e-23,[Metabolism of RNA]


Number of filtered terms: 3
Size of community: 1181


,Term,Overlap,Adjusted P-value,Level_1_Categories
0,RAB Geranylgeranylation R-HSA-8873719,17/62,0.000035,[Metabolism of proteins]
1,Rab Regulation Of Trafficking R-HSA-9007101,24/122,0.000042,[Vesicle-mediated transport]
2,Membrane Trafficking R-HSA-199991,64/599,0.000534,[Vesicle-mediated transport]


Number of filtered terms: 0
Size of community: 720


...

,Term,Overlap,Adjusted P-value,Level_1_Categories


Number of filtered terms: 2
Size of community: 61


,Term,Overlap,Adjusted P-value,Level_1_Categories
0,Expression And Translocation Of Olfactory Receptors R-HSA-9752946,61/393,1.839754e-106,[Sensory Perception]
1,Olfactory Signaling Pathway R-HSA-381753,61/401,3.489023e-106,[Sensory Perception]


Number of filtered terms: 0
Size of community: 51


,Term,Overlap,Adjusted P-value,Level_1_Categories


### Disease Data Sets

In [ ]:
disease_size_cap = 200
disease_score_cap = 0.0001
important_diseases = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value"])

In [ ]:
# Disease-gene enrichment libraries
disease_sets = [
    'DisGeNET_2020', # curated gene–disease associations
    'GWAS_Catalog_2023', # genome-wide association hits
    'OMIM_Disease', # Mendelian disorders
    'Jensen_DISEASES' # text-mined associations
]

# # Disease-gene enrichment Analysis; save terms with small size and high p-value
i = 0
for community in communities_HGNC:
    # Gene Ontology enrichment
    enr_disease = gp.enrichr(
        gene_list=community,
        gene_sets=disease_sets,
        organism='Human',
        outdir=None # don't write to disk
    )
    enr_disease_df = enr_disease.results.sort_values('Adjusted P-value')
    print(f"Size of community: {len(community)}")

    mask = ((enr_disease_df["Overlap"].apply(lambda x: int(x.split("/")[1]) < disease_size_cap)) 
    & (enr_disease_df["Adjusted P-value"] < disease_score_cap))
        
    filtered = enr_disease_df[mask].copy()
    if not filtered.empty:
        filtered.loc[:, "Community Index"] = i
        filtered.loc[:, "Community Size"] = len(community)
        important_diseases = pd.concat([important_diseases, filtered], ignore_index=True)

    display(HTML(enr_disease_df[['Term','Overlap','Adjusted P-value']].head(10).to_html(max_cols=None)))
    i += 1

# Important Terms Analysis

In [ ]:
important_terms = important_terms.sort_values(by="Overlap")
display(HTML(important_terms[["Community Index","Community Size",'Term','Overlap','Adjusted P-value',"P-value"]].head(50).to_html(max_cols=None)))

# Important Diseases Analysis

In [ ]:
important_diseases = important_diseases.sort_values(by="Community Index")
display(HTML(important_diseases[["Community Index","Community Size",'Term','Overlap','Adjusted P-value']].head(50).to_html(max_cols=None)))